# Visualizing Kuzu Databases <a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs-for-kuzu/blob/kuzu/examples/introduction.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates how to visualize Cypher queries on [Kuzu databases](https://kuzudb.com) using `yfiles-jupyter-graphs-for-kuzu`.

## Install required dependencies

In [ ]:
%pip install kuzu --quiet
%pip install yfiles_jupyter_graphs_for_kuzu --quiet

You can also open this notebook in Google Colab when Google Colab's custom widget manager is enabled:

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

## Download sample data

For demonstration purpose, download some sample data from [kuzudb.com](https://kuzudb.com) and create a local database with it.

In [ ]:
import requests

urls = [
    "https://kuzudb.com/data/movie-lens/movies.csv",
    "https://kuzudb.com/data/movie-lens/users.csv",
    "https://kuzudb.com/data/movie-lens/ratings.csv",
    "https://kuzudb.com/data/movie-lens/tags.csv"
]

for url in urls:
    filename = url.split("/")[-1]
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {filename}")

## Create sample database

Use the dowloaded CSV files to create a sample database.

In [ ]:
import shutil
import kuzu

db_path = './_db_ml-small'
shutil.rmtree(db_path, ignore_errors=True)

def load_data(connection):
    connection.execute('CREATE NODE TABLE Movie (movieId INT64, year INT64, title STRING, genres STRING, PRIMARY KEY (movieId))')
    connection.execute('CREATE NODE TABLE User (userId INT64, PRIMARY KEY (userId))')
    connection.execute('CREATE REL TABLE Rating (FROM User TO Movie, rating DOUBLE, timestamp INT64)')
    connection.execute('CREATE REL TABLE Tags (FROM User TO Movie, tag STRING, timestamp INT64)')

    connection.execute('COPY Movie FROM "./movies.csv" (HEADER=TRUE)')
    connection.execute('COPY User FROM "./users.csv" (HEADER=TRUE)')
    connection.execute('COPY Rating FROM "./ratings.csv" (HEADER=TRUE)')
    connection.execute('COPY Tags FROM "./tags.csv" (HEADER=TRUE)')

db = kuzu.Database(db_path)
conn = kuzu.Connection(db)
load_data(conn)

# Visualize queries with `yfiles-jupyter-graphs-for-kuzu`

In [ ]:
from yfiles_jupyter_graphs_for_kuzu import KuzuGraphWidget
g = KuzuGraphWidget(conn)

In [ ]:
g.show_cypher('MATCH (u:User)-[r:Rating]->(m:Movie) RETURN u, r, m LIMIT 250')

# Customize visualization by providing different node/relationship mappings

In [ ]:
g = KuzuGraphWidget(conn)
g.add_node_configuration('Movie', text= lambda node : {
        'text': 'Title:\n' + node['properties']['title'], 
        'backgroundColor': 'rgba(0,0,0,0.7)', 
        'fontSize': 20, 
        'color': '#FFFFFF', 
        'position': 'south', 
        'fontWeight': 'lighter',
        'maximumWidth': 300, 
        'wrapping': 'word', 
        'textAlignment': 'center'
    }, parent_configuration='movies')  
g.show_cypher('MATCH (u:User)-[r:Rating]->(m:Movie) RETURN u, r, m LIMIT 250', layout="hierarchic")